In [1]:
!nvidia-smi

Mon Feb 21 14:41:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install -q transformers datasets torchinfo rouge_score git+https://github.com/google-research/bleurt.git

In [1]:
from google.colab import drive

drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [14]:
RANDOM_SEED = 42

# Dataset loading

In [ ]:
# load already tokenized dataset
from datasets import load_dataset, load_from_disk
DATASET_PATH = "/gdrive/MyDrive/university/tokenized_dataset"
dataset = load_from_disk(GDRIVE_DATASET_PATH)

# split dataset
train_test = dataset.train_test_split(test_size=0.15, seed=RANDOM_SEED)
test_valid = train_test["train"].train_test_split(test_size=0.1, seed=RANDOM_SEED)
train_test["train"] = test_valid["train"]
train_test["valid"] = test_valid["test"]
dataset = train_test

# delete from memory unused values
del train_test
del test_valid

# Model loading

In [ ]:
from transformers import BigBirdPegasusForConditionalGeneration
from torchinfo import summary

MODEL_PATH = "google/bigbird-pegasus-large-bigpatent"

model = BigBirdPegasusForConditionalGeneration.from_pretrained(
    "google/bigbird-pegasus-large-bigpatent",
    block_size=16,
    num_random_blocks=3,
    attention_type="block_sparse",
    use_cache=False) # required for fp16
model.gradient_checkpointing_enable()
summary(model, dtypes=["torch.IntTensor"])

# Generation

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-bigpatent")
valid_sample = dataset["valid"]["input_ids"][0]

# print summary
tokenizer.decode(valid_sample, skip_special_tokens=True)

In [ ]:
# print claim
tokenizer.decode(dataset["valid"]["decoder_input_ids"][0], skip_special_tokens=True)

In [ ]:
import torch
valid_sample = torch.tensor(valid_sample)
inputs = tokenizer([tokenizer.decode(valid_sample, skip_special_tokens=True)], 
                   max_length=2048, 
                   return_tensors="pt", 
                   truncation=True)

In [ ]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    inputs["input_ids"], 
    max_length=50, 
    num_beams=5, 
    #no_repeat_ngram_size=2,
    repetition_penalty=0.5,
    #num_return_sequences=5, 
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))